<a href="https://colab.research.google.com/github/MS-H2020/Open/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [プリント基板の電子部品検出 Nishika株式会社](https://competition.nishika.com/competitions/kiban/summary)

## Build Environment

/content/drive/MyDrive/PCB_Detect/: Working directory  
┣ 01_input: A directory includes input data in competition  
┃ ┣ train_imgs.zip: A zip file including train images    
┃ ┣ test_imgs.zip: A zip file including test images  
┃ ┣ train.csv:  A csv file in which table data written for training.   
┃ ┣ test.csv: A csv file in which table data written for test.  
┃ ┗ sample_submission.csv: A csv file is sample for submission.  
┣ 03_code: A directory for main code.
┣ 04_model: A directory for model.  
┗ 05_submission: A directory for submission data file.  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
TEMP_DIR = "./temp"
INPUT_DIR:str = "/content/drive/MyDrive/PCB_Detect/01_input"
#TRAIN_DIR:str = INPUT_DIR + "/train_data"
#TEST_DIR:str = INPUT_DIR + "/test_data"
#ANALYSIS_DIR:str = "/content/drive/MyDrive/Human_Activity_Recognition_with_Smartphones/03_code-analysis"
MODEL_DIR:str = "/content/drive/MyDrive/PCB_Detect//04_model"
SUBMISSION_DIR:str = '/content/drive/MyDrive/PCB_Detect/05_submission'
CHECKPOINT_PATH:str = MODEL_DIR

In [9]:
import os, shutil, glob, zipfile

if not os.path.isdir('./train_imgs') and  not os.path.isdir('./test_imgs'):
  # Copy ZIP files including images, and Unzip them.
  !cp -r "/content/drive/MyDrive/PCB_Detect/01_input/train_imgs.zip" ./
  !cp -r "/content/drive/MyDrive/PCB_Detect/01_input/test_imgs.zip" ./
  shutil.unpack_archive('train_imgs.zip', './')
  shutil.unpack_archive('test_imgs.zip', './')

## Define Rle decoding with tensorflow

In [6]:
import tensorflow as tf

def rle_decode_tf(mask_rle, shape):
    shape = tf.convert_to_tensor(shape, tf.int64)
    size = tf.math.reduce_prod(shape)
    # Split string
    s = tf.strings.split(mask_rle)
    s = tf.strings.to_number(s, tf.int64)
    # Get starts and lengths
    starts = s[::2] - 1
    lens = s[1::2]
    # Make ones to be scattered
    total_ones = tf.reduce_sum(lens)
    ones = tf.ones([total_ones], tf.uint8)
    # Make scattering indices
    r = tf.range(total_ones)
    lens_cum = tf.math.cumsum(lens)
    s = tf.searchsorted(lens_cum, r, 'right')
    idx = r + tf.gather(starts - tf.pad(lens_cum[:-1], [(1, 0)]), s)
    # Scatter ones into flattened mask
    mask_flat = tf.scatter_nd(tf.expand_dims(idx, 1), ones, [size])
    # Reshape into mask
    return tf.reshape(mask_flat, shape)